In [ ]:
import requests
import json
with open('./DataAnnotation/gpt-4o-errors.json','r') as f:
    all_data = json.load(f)
for element in all_data:
        system_prompt_list = [
            """
            Role: 高级代码审查专家
            Profile
            description: 担任资深代码审查专家，负责对代码进行人工按顺序走查，识别潜在的安全缺陷，
            并给出具体的CWE类型，直接给出规定格式的结果，不需要额外的解释。
                """,
                """
                Role: 高级代码审查专家
                Profile
                description: 担任资深代码审查专家，负责对代码进行人工按顺序走查，识别潜在的安全缺陷，
                并给出具体的CWE类型，直接给出规定格式的结果，不需要额外的解释。

                Skills
                - 熟悉常见的软件安全漏洞和CWE (Common Weakness Enumeration) 列表
                - 具备代码静态分析和人工代码走查的能力
                - 能够识别多种编程语言的潜在安全风险
                - 能够清晰地描述安全缺陷并给出CWE类型
                - 能够理解并应用安全编程最佳实践

                Background:
                - 具备多年软件开发和代码安全审查经验
                - 熟悉常见的软件安全标准和规范

                Constraints
                - 严格按照代码执行顺序进行审查
                - 审查报告需清晰简洁，避免使用模糊的描述
                - 必须充分理解代码逻辑，避免误判和漏判
                """,
                """
                Role: 资深的代码修复专家

                Profile
                - description: 你是一名资深的代码修复专家，你需要根据上文提供的代码、代码功能以及代码缺陷CWE类型，对代码中存在的CWE类型进行修复。

                Skills

                - 精通各种编程语言，包括但不限于C/C++、Java、Python、JavaScript等。
                - 深入理解各种常见的代码缺陷类型（CWE），并能准确识别代码中存在的安全漏洞。
                - 能够根据代码功能、上下文和CWE类型，提出有效的代码修复方案。
                - 能够清晰地解释代码缺陷产生的原因和修复方案的原理。
                - 能够编写高质量、安全的代码，并进行充分的测试验证。

                Goals

                - 准确识别并修复代码中存在的指定CWE类型的缺陷。
                - 确保修复后的代码能够正常实现原有功能。
                - 提供清晰的修复说明和建议，帮助开发者理解并避免类似问题。

                Constraints

                - 修复方案必须针对指定的CWE类型。
                - 修复后的代码必须保持原有的编程语言和代码风格。
                - 必须清晰解释修复思路和原理。
                """
        ]
        history = [
                {"role": "user", "content": f"请仔细阅读以下代码，请用不超过三十个字来描述这段代码的功能:\n{element['question']}"}
        ]
        count = 0
        prompt = [{"role": "system", "content": system_prompt_list[count]}] + history
        count += 1
        function_description = element['function_description']
        history.append({"role": "assistant", "content": f"{function_description}"})
        analysis_prompt_temp = [
                {"role": "user", "content": f"""请根据上述代码及信息，是否存在任何潜在的软件漏洞或缺陷？如果有，请详细描述漏洞的成因，并尝试指出可能的CWE类型（CWE Top10CWE-other），缺陷原因，产生缺陷的代码【可能存在多行代码】，CWE-Top10解释如下：重点关注以及CWE-787（越界写入）、CWE-79（在Web页面生成时对输入的转义处理不恰当）、CWE-89（SQL注入问题）、CWE-416（内存安全：释放后使用）、CWE-78（OS命令注入）、CWE-20（不正确的输入验证）、CWE-125（越界读取）、CWE-22（文件处理；对路径名的限制不恰当）、CWE-352（数据真实性验证不足；跨站请求伪造）、CWE-434（危险类型文件的不加限制上传）。
                请按照如下格式给出答案：
                如果代码中不存在缺陷，则输出：{{'CWE_Type':'pass!'}}\n
                如果代码中存在缺陷，则输出：\n
                [
                {{'CWE_Type':''# 指CWE类型, 'CWE_Code':''# 指具有缺陷的代码, 'CWE_Description':''# 中文解释}},\n
                {{'CWE_Type':''# 指CWE类型, 'CWE_Code':''# 指具有缺陷的代码, 'CWE_Description':''# 中文解释}}
                ]"""
                }
        ]
        history.extend(analysis_prompt_temp)
        prompt = [{"role": "system", "content": system_prompt_list[count]}] + history
        count += 1
        vulnerability_analysis = element['vulnerability_analysis']
        history.append({"role": "assistant", "content": f"{vulnerability_analysis}"})
        repair_prompt_temp = [
                {"role": "user", "content": f"""
                基于以上的代码漏洞分析和代码，给出修改之后的代码（主要是完整代码），并给出简要的修复方法(指你是如何修复的)，不需要再进行额外的解释。
                请你按照如下格式给出答案：
                [
                {{'repair_code':'',  # 修复后完整的代码
                'repair_method':'' # 中文解释
                }},\n
                ]
                """}
        ]
        history.extend(repair_prompt_temp)
        prompt = [{"role": "system", "content": system_prompt_list[count]}] + history

        headers = {
                'Content-Type': 'application/json',
                'Authorization': f'Bearer sk-xXMFNG7h4SNXqOOw6KfeRm4zNfK6euk9DMsuHAlbLBcMEczm'
        }
        data = {
                "model": 'gpt-4o',
                "messages": prompt,
                "temperature": 0.00
        }
        response = requests.post('https://fastapi.aabao.top/v1/chat/completions', headers=headers, json=data, stream=True)
        element['repaired_code'] = response.json()['choices'][0]['message']['content'].replace('\n', '\\n')

In [7]:
all_data[0]

{'question': 'private void loadGlobalSettings(android.database.sqlite.SQLiteDatabase db) {\n    android.database.sqlite.SQLiteStatement stmt = null;\n    try {\n        stmt = db.compileStatement(("INSERT OR IGNORE INTO global(name,value)" + " VALUES(?,?);"));\n        loadBooleanSetting(stmt, Settings.Global.AIRPLANE_MODE_ON, R.bool.def_airplane_mode_on);\n        loadStringSetting(stmt, Settings.Global.AIRPLANE_MODE_RADIOS, R.string.def_airplane_mode_radios);\n        loadStringSetting(stmt, Settings.Global.AIRPLANE_MODE_TOGGLEABLE_RADIOS, R.string.airplane_mode_toggleable_radios);\n        loadBooleanSetting(stmt, Settings.Global.ASSISTED_GPS_ENABLED, R.bool.assisted_gps_enabled);\n        loadBooleanSetting(stmt, Settings.Global.AUTO_TIME, R.bool.def_auto_time);\n        loadBooleanSetting(stmt, Settings.Global.AUTO_TIME_ZONE, R.bool.def_auto_time_zone);\n        loadSetting(stmt, Settings.Global.STAY_ON_WHILE_PLUGGED_IN, (("1".equals(android.os.SystemProperties.get("ro.kernel.qemu

In [ ]:
with open('./Annotation/results/split2_output_gpt--4o.json', 'r') as f:
    original_data = json.load(f)
for element in all_data:
    for item in original_data:
        if item['question'] == element['question']:
            item['repaired_code'] = element['repaired_code']
with open('./Annotation/results/split2_output_gpt--4o_all.json', 'w') as f:
    json.dump(original_data, f, indent=4, ensure_ascii=False)


In [16]:
print(response.json()['choices'][0]['message']['content'])

[
    {'repair_code': '''
#include <bits/stdc++.h>

using namespace std;

#define FAST ios_base::sync_with_stdio(false); cin.tie(NULL); cout.tie(NULL);
#define ll long long
#define ull unsigned long long
#define pi 2*acos(0.0)
#define readl(v,n) for(ll i=0;i<n;i++) {ll val; cin>>val; v.pb(val);}
#define readi(v,n) for(int i=0;i<n;i++) {int val; cin>>val; v.pb(val);}
#define srt(v) sort(v.begin(), v.end());
#define rsrt(v) sort(v.rbegin(), v.rend());
#define MIN(v) *min_element(v.begin(), v.end())
#define MAX(v) *max_element(v.begin(), v.end())
#define sz(x) ((ll) (x).size())
#define all(x) (x).begin(), (x).end()
#define rep(i, a, b) for(ll i = (a); i < (b); i++)
#define rep2(i, a, b) for(ll i = (a); i <= (b); i++)
#define vll vector <ll>
#define vii vector <int>
#define pii pair <int, int>
#define pll pair <ll, ll>
#define M 1000007
#define MOD 1000000007
#define pb push_back
#define mp make_pair
#define ff first
#define ss second
#define endl "\\n"
#define inf 1000000000000000000
#def

In [17]:
# 将response.json()['choices'][0]['message']['content']转成string，保留换行符
print(response.json()['choices'][0]['message']['content'].replace('\n', '\\n'))

[\n    {'repair_code': '''\n#include <bits/stdc++.h>\n\nusing namespace std;\n\n#define FAST ios_base::sync_with_stdio(false); cin.tie(NULL); cout.tie(NULL);\n#define ll long long\n#define ull unsigned long long\n#define pi 2*acos(0.0)\n#define readl(v,n) for(ll i=0;i<n;i++) {ll val; cin>>val; v.pb(val);}\n#define readi(v,n) for(int i=0;i<n;i++) {int val; cin>>val; v.pb(val);}\n#define srt(v) sort(v.begin(), v.end());\n#define rsrt(v) sort(v.rbegin(), v.rend());\n#define MIN(v) *min_element(v.begin(), v.end())\n#define MAX(v) *max_element(v.begin(), v.end())\n#define sz(x) ((ll) (x).size())\n#define all(x) (x).begin(), (x).end()\n#define rep(i, a, b) for(ll i = (a); i < (b); i++)\n#define rep2(i, a, b) for(ll i = (a); i <= (b); i++)\n#define vll vector <ll>\n#define vii vector <int>\n#define pii pair <int, int>\n#define pll pair <ll, ll>\n#define M 1000007\n#define MOD 1000000007\n#define pb push_back\n#define mp make_pair\n#define ff first\n#define ss second\n#define endl "\\n"\n#defi

In [ ]:
import json
with open('./Annotation/results/split2_random_model_sample_output_with_fixed_code.json', 'r') as f:
    data1 = json.load(f)
for item in data1:
    if item['question'] == dt['question']:
        item['repaired_code'] = response.json()['choices'][0]['message']['content'].replace('\n', '\\n')
with open('./Annotation/results/split2_random_model_sample_output_with_fixed_code.json', 'w') as f:
    json.dump(data1, f, indent=4, ensure_ascii=False)


In [19]:
data

{'model': 'gpt-4o',
 'messages': [{'role': 'system',
   'content': '\n        Role: 资深的代码修复专家\n\n        Profile\n        - description: 你是一名资深的代码修复专家，你需要根据上文提供的代码、代码功能以及代码缺陷CWE类型，对代码中存在的CWE类型进行修复。\n\n        Skills\n\n        - 精通各种编程语言，包括但不限于C/C++、Java、Python、JavaScript等。\n        - 深入理解各种常见的代码缺陷类型（CWE），并能准确识别代码中存在的安全漏洞。\n        - 能够根据代码功能、上下文和CWE类型，提出有效的代码修复方案。\n        - 能够清晰地解释代码缺陷产生的原因和修复方案的原理。\n        - 能够编写高质量、安全的代码，并进行充分的测试验证。\n\n        Goals\n\n        - 准确识别并修复代码中存在的指定CWE类型的缺陷。\n        - 确保修复后的代码能够正常实现原有功能。\n        - 提供清晰的修复说明和建议，帮助开发者理解并避免类似问题。\n\n        Constraints\n\n        - 修复方案必须针对指定的CWE类型。\n        - 修复后的代码必须保持原有的编程语言和代码风格。\n        - 必须清晰解释修复思路和原理。\n        '},
  {'role': 'user',
   'content': '请仔细阅读以下代码，请用不超过三十个字来描述这段代码的功能:\n#include <bits/stdc++.h>\r\n\r\nusing namespace std;\r\n\r\n#define FAST ios_base::sync_with_stdio(false); cin.tie(NULL); cout.tie(NULL);\r\n#define ll long long\r\n#define ull unsigned long long\r\n#define pi 2*acos(0.0)\r\n#define readl

In [21]:
dt = {
        "question": "#include <bits/stdc++.h>\r\n\r\nusing namespace std;\r\n\r\n#define FAST ios_base::sync_with_stdio(false); cin.tie(NULL); cout.tie(NULL);\r\n#define ll long long\r\n#define ull unsigned long long\r\n#define pi 2*acos(0.0)\r\n#define readl(v,n) for(ll i=0;i<n;i++) {ll val; cin>>val; v.pb(val);}\r\n#define readi(v,n) for(int i=0;i<n;i++) {int val; cin>>val; v.pb(val);}\r\n#define srt(v) sort(v.begin(), v.end());\r\n#define rsrt(v) sort(v.rbegin(), v.rend());\r\n#define MIN(v) *min_element(v.begin(), v.end())\r\n#define MAX(v) *max_element(v.begin(), v.end())\r\n#define sz(x) ((ll) (x).size())\r\n#define all(x) (x).begin(), (x).end()\r\n#define rep(i, a, b) for(ll i = (a); i < (b); i++)\r\n#define rep2(i, a, b) for(ll i = (a); i <= (b); i++)\r\n#define vll vector <ll>\r\n#define vii vector <int>\r\n#define pii pair <int, int>\r\n#define pll pair <ll, ll>\r\n#define M 1000007\r\n#define MOD 1000000007\r\n#define pb push_back\r\n#define mp make_pair\r\n#define ff first\r\n#define ss second\r\n#define endl \"\\n\"\r\n#define inf 1000000000000000000\r\n#define ninf -1000000000000000000\r\n#define deb(n) cout<<\"--\"<<(n)<<\"--\"<<endl\r\n#define debx(x) cout << #x << \" = \" << (x) << endl\r\n#define deb2(a,b) cout<<(a)<<\"----\"<<(b)<<endl\r\n#define debv(v) for(auto x: v) {cout<<x<<\", \";} cout<<endl;\r\n#define put(n) cout<<(n)<<endl\r\n#define toDecimal(s) bitset<64>(s).to_ullong()\r\n#define toBinaryStr(n) bitset<64>(n).to_string()\r\n#define toBinary(n) stoll(bitset<64>(n).to_string())\r\n\r\n/*\r\n    ---- take notes here ----\r\n    \r\n*/\r\n\r\nvoid solve()\r\n{\r\n    ll n,k; cin>>n>>k;\r\n    ll x=0;\r\n    ll r=0;\r\n    while(!r)\r\n    {\r\n        cout<<x<<endl;\r\n        cout.flush();\r\n        if(x==1) x=n-1;\r\n        else x/=2;\r\n    }\r\n}\r\n\r\nint main()\r\n{\r\n    FAST\r\n    int t = 1;\r\n    cin >> t;\r\n    while(t--)\r\n    {\r\n        solve();    \r\n    }\r\n\r\n    return 0;\r\n}",
        "function_description": "该代码实现的是进行t次循环的文本输出。",
        "vulnerability_analysis": "[\n    {'CWE_Type':'CWE-20', 'CWE_Code':'while(!r)', 'CWE_Description':'不正确的输入验证导致死循环，因为r值没有得到更新。'}\n]",
        "repaired_code": "[\n    {\n        'repair_code':'''\n#include <bits/stdc++.h>\n\nusing namespace std;\n\n#define FAST ios_base::sync_with_stdio(false); cin.tie(NULL); cout.tie(NULL);\n#define ll long long\n#define ull unsigned long long\n#define pi 2*acos(0.0)\n#define readl(v,n) for(ll i=0;i<n;i++) {ll val; cin>>val; v.pb(val);}\n#define readi(v,n) for(int i=0;i<n;i++) {int val; cin>>val; v.pb(val);}\n#define srt(v) sort(v.begin(), v.end());\n#define rsrt(v) sort(v.rbegin(), v.rend());\n#define MIN(v) *min_element(v.begin(), v.end())\n#define MAX(v) *max_element(v.begin(), v.end())\n#define sz(x) ((ll) (x).size())\n#define all(x) (x).begin(), (x).end()\n#define rep(i, a, b) for(ll i = (a); i < (b); i++)\n#define rep2(i, a, b) for(ll i = (a); i <= (b); i++)\n#define vll vector <ll>\n#define vii vector <int>\n#define pii pair <int, int>\n#define pll pair <ll, ll>\n#define M 1000007\n#define MOD 1000000007\n#define pb push_back\n#define mp make_pair\n#define ff first\n#define ss second\n#define endl \"\\\\n\"\n#define inf 1000000000000000000\n#define ninf -1000000000000000000\n#define deb(n) cout<<\"--\"<<(n)<<\"--\"<<endl\n#define debx(x) cout << #x << \" = \" << (x) << endl\n#define deb2(a,b) cout<<(a)<<\"----\"<<(b)<<endl\n#define debv(v) for(auto x: v) {cout<<x<<\", \";} cout<<endl;\n#define put(n) cout<<(n)<<endl\n#define toDecimal(s) bitset<64>(s).to_ullong()\n#define toBinaryStr(n) bitset<64>(n).to_string()\n#define toBinary(n) stoll(bitset<64>(n).to_string())\n\nvoid solve()\n{\n    ll n, k; cin >> n >> k;\n    ll x = 0;\n    ll r = 0; // initially 0, modify if needed during iterations\n    while (r == 0) // updated condition for improved clarity\n    {\n        cout << x << endl;\n        cout.flush();\n        if (x == 1) \n         .\n          try also check.Suspend parsing",
        "llm_model_first_step": "gpt-4o",
        "llm_model_second_step": "gpt-4o",
        "llm_model_third_step": "gpt-4o",
        "source": "C++_3409220.json",
    }